In [ ]:
!pip install -q kaggle

In [ ]:
!wget 'https://raw.githubusercontent.com/kwanhong66/KaggleShoveling/master/token/kaggle.json'

--2024-05-06 17:32:01--  https://raw.githubusercontent.com/kwanhong66/KaggleShoveling/master/token/kaggle.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63 [text/plain]
Saving to: ‘kaggle.json.2’

kaggle.json.2       100%[===================>]      63  --.-KB/s    in 0s      

2024-05-06 17:32:01 (4.17 MB/s) - ‘kaggle.json.2’ saved [63/63]



In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!unzip '*.zip' -d ./input/

Archive:  jigsaw-toxic-comment-classification-challenge.zip
replace ./input/sample_submission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./input/sample_submission.csv.zip  
replace ./input/test.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./input/test.csv.zip    
replace ./input/test_labels.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./input/test_labels.csv.zip  
replace ./input/train.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./input/train.csv.zip   


In [ ]:
!kaggle competitions download jigsaw-toxic-comment-classification-challenge

jigsaw-toxic-comment-classification-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!mkdir input

mkdir: cannot create directory ‘input’: File exists


In [ ]:
!unzip '/content/input/train.csv.zip' -d ./input/
!unzip '/content/input/test.csv.zip' -d ./input
!unzip '/content/input/sample_submission.csv.zip' -d ./input
!unzip '/content/input/test_labels.zip' -d ./input/
!unzip '/content/input/test_labels.csv.zip' -d ./input/

Archive:  /content/input/train.csv.zip
  inflating: ./input/train.csv       
Archive:  /content/input/test.csv.zip
  inflating: ./input/test.csv        
Archive:  /content/input/sample_submission.csv.zip
  inflating: ./input/sample_submission.csv  
unzip:  cannot find or open /content/input/test_labels.zip, /content/input/test_labels.zip.zip or /content/input/test_labels.zip.ZIP.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_csv('/content/input/train.csv')
test_df = pd.read_csv('/content/input/test.csv')
test_labels_df = pd.read_csv('/content/input/test_labels.csv')

In [ ]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
test_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [ ]:
test_labels_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [ ]:
labels = train_df.columns[2:].to_numpy()
labels

array(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'], dtype=object)

In [ ]:
train_sentences = train_df["comment_text"]
train_labels = train_df.iloc[:, 2:]

print(f"Train sentences: {train_sentences.shape}")
print(f"Train labels: {train_labels.shape}")

Train sentences: (159571,)
Train labels: (159571, 6)


In [ ]:
from sklearn.model_selection import train_test_split

train_pct = 0.8

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_sentences,
    train_labels,
    train_size=0.8
)

print(f"Train sentences: {train_sentences.shape}")
print(f"Train labels: {train_labels.shape}")
print(f"Validation sentences: {val_sentences.shape}")
print(f"Validation labels: {val_labels.shape}")

Train sentences: (127656,)
Train labels: (127656, 6)
Validation sentences: (31915,)
Validation labels: (31915, 6)


In [ ]:
import string

def remove_punctuation(text):
    return text.translate(str.maketrans("", "", string.punctuation))

def remove_special_chars(text):
    return text.replace("\n", " ").replace("\t", " ").strip()

def normalize_spacing(text):
    return " ".join(text.split())

def process_text(text):
    text = remove_punctuation(text)
    text = remove_special_chars(text)
    text = normalize_spacing(text)
    return text

In [ ]:
train_sentences = train_sentences.map(process_text)
val_sentences = val_sentences.map(process_text)

In [ ]:
train_sentences = train_sentences.to_numpy()
train_labels = train_labels.to_numpy()
val_sentences = val_sentences.to_numpy()
val_labels = val_labels.to_numpy()

In [ ]:
train_sentences

array(['Roux I think you should get away from this page Your acting like some vandal or something Alot of people contribute to this page and you seem to want to delete EVERYONES work The reality is there is very little documentation on Trance music in texbooks This poses a problem In this scenario the Wiki relies on the followers of the genre to input their expertise FROM FIRST HAND EXPERIENCE If this werent the case the page would be almost absolutely empty with nothing in it I know you like to say there are rules for this and that but in this case because there is no documented research it is all we can work with Be happy with what is provided Lastly the removing of Discogs as a reference is the most bizarre thing Ive ever heard Trance music is about the musicand it was experienced through records NOT BOOKS This in of itself is the most important reference of them all',
       'Thank you for the change Western province is still missing',
       'sorry not directed at me but i dont wa

In [ ]:
vocab = set()
for sentence in train_sentences:
    for word in sentence.split():
        vocab.add(word)

vocab_size = len(vocab)
print(f"Vocab size: {vocab_size}")

Vocab size: 261103


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_sentences)

In [ ]:
import json

vocab = {}
for word, index in tokenizer.word_index.items():
    if index <= vocab_size:
        vocab[word] = index

print(len(vocab))

with open("tokenizer_dictionary.json", "w") as file:
    json.dump(vocab, file)

221455


In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

max_length = 300
padding_type = "post"
trunc_type = "post"

train_sequences = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
val_sequences = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(f"Train sentences (vectorized): {train_sequences.shape}")
print(f"Validation sentences (vectorized): {val_sequences.shape}")

Train sentences (vectorized): (127656, 300)
Validation sentences (vectorized): (31915, 300)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, TextVectorization, Input
from tensorflow import keras

In [ ]:
embedding_dim = 200

In [ ]:
model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(16, activation="tanh"),
    Dense(6, activation="sigmoid"),
])

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
epochs = 2
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3),
]

In [ ]:
model.evaluate(val_sequences, val_labels)

998/998 [==============================] - 31s 31ms/step - loss: 0.1386 - accuracy: 0.9936


[0.13863582909107208, 0.99357670545578]

In [ ]:
model.fit(
    train_sequences,
    train_labels,
    validation_data=(val_sequences, val_labels),
    epochs=epochs,
    batch_size=200,
    callbacks=callbacks,
)

Epoch 1/2
639/639 [==============================] - 1211s 2s/step - loss: 0.1403 - accuracy: 0.9922 - val_loss: 0.1386 - val_accuracy: 0.9888
Epoch 2/2
639/639 [==============================] - 1203s 2s/step - loss: 0.1399 - accuracy: 0.9934 - val_loss: 0.1384 - val_accuracy: 0.9937


In [ ]:
def predict(sentence):
    # convert sentence to sequence
    sequences = tokenizer.texts_to_sequences([sentence])
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    # get predictions for toxicity
    predictions = model.predict(padded_sequences)[0]

    return list(zip(labels, predictions))

In [ ]:
predict("I hate you")

1/1 [==============================] - 1s 736ms/step


[('toxic', 0.09413036),
 ('severe_toxic', 0.008987832),
 ('obscene', 0.052967623),
 ('threat', 0.002699154),
 ('insult', 0.048933554),
 ('identity_hate', 0.0086317565)]

In [ ]:
predict("I love you")

1/1 [==============================] - 0s 65ms/step


[('toxic', 0.09413036),
 ('severe_toxic', 0.008987832),
 ('obscene', 0.052967623),
 ('threat', 0.002699154),
 ('insult', 0.048933554),
 ('identity_hate', 0.0086317565)]

In [ ]:
loss, accuracy = model.evaluate(val_sequences, val_labels)
print(f"Accuracy on validation sentences: {accuracy}")

998/998 [==============================] - 31s 31ms/step - loss: 0.1384 - accuracy: 0.9937
Accuracy on validation sentences: 0.9937019944190979
